In [3]:
import cobra
import os
import efmtool
import numpy as np
import pandas as pd
import tqdm

In [4]:
model=cobra.io.read_sbml_model('../../Model/iCH360red/iCH360red.xml')

In [5]:
def count_flux_supporting(efm_df,fluxes:list):
    satisfied=np.ones(efm_df.shape[1])
    for flux in fluxes:
        satisfied=np.logical_and(satisfied,efm_df.loc[flux].to_numpy())
    candidates=np.nonzero(satisfied)[0]
    return candidates

def enumerate_efms(model,
                carbon_source,
                condition,
                fluxes_to_enforce=['Biomass'],
                carbon_source_specific_remove=None,
                condition_specific_remove=None,
                out_path='./efms'):
    with model as m:
        m.reactions.EX_glc__D_e.bounds=(0,1000)
        m.reactions.get_by_id(f'EX_{carbon_source}_e').lower_bound=-10
        if condition=='aerobic':
            m.reactions.get_by_id('EX_o2_e').lower_bound=-1000
        elif condition=='anaerobic':
            m.reactions.get_by_id('EX_o2_e').lower_bound=0

        else:
            raise ValueError
        print(f'Computing EFMs for {carbon_source} {condition}....')
        original_rxn_ixs=[r.id for r in model.reactions]
        removed_ids=[]
        #Remove carbon source specific reactions
        if carbon_source_specific_remove is not None:
            for cs in carbon_source_specific_remove.keys():
                if carbon_source==cs:
                    removed_rxns=[m.reactions.get_by_id(r) for r in carbon_source_specific_remove[cs] if r in model.reactions]
                    removed_ids+=[r.id for r in removed_rxns]
                    m.remove_reactions(removed_rxns)
        #Remove condition-specific reactions
        if condition_specific_remove is not None:
            for cond in condition_specific_remove.keys():
                if condition==cond:
                    removed_rxns=[m.reactions.get_by_id(r) for r in condition_specific_remove[cond] if r in model.reactions]
                    removed_ids+=[r.id for r in removed_rxns]
                    m.remove_reactions(removed_rxns)
        print(f'Removed {removed_ids} due to carbon source or condition specific parameter.')
        reversibilities=[int(r.reversibility) for r in m.reactions]
        reactions_ids=[r.id for r in m.reactions]
        metabolite_ids=[m.id for m in m.metabolites]
        S=cobra.util.array.create_stoichiometric_matrix(m,array_type='DataFrame')
        S=S.values


        filename_csv=f'ich360red_{carbon_source}_{condition}_efms.csv'
        path_csv=os.path.join(out_path,filename_csv)
        
        
        
        print('Starting EFMTools computation') 
        efm_options={
                "kind": "stoichiometry",
                "arithmetic": "fractional",
                "zero": "1e-12",
                "compression": "default",
                "log": "console",
                "level": "INFO",
                "maxthreads": "-1",
                "normalize": "min",
                "adjacency-method": "pattern-tree-minzero",
                "rowordering": "MostZerosOrAbsLexMin",
            }
        
        
        efms=efmtool.calculate_efms(stoichiometry=S,
                                reversibilities=reversibilities,
                                reaction_names=reactions_ids,
                                metabolite_names=metabolite_ids,
                                options=efm_options)
        
        efms_df=pd.DataFrame(data=efms,index=reactions_ids,columns=list(range(efms.shape[1])))
        #If we have removed any reaction because of condition specific simplifications, let's add them back with 0 flux for consistency
        if removed_ids:
            zero_efm_rows=pd.DataFrame(index=removed_ids,
                                    columns=efms_df.columns,
                                    data=np.zeros((len(removed_ids),efms.shape[1]))
                                    )
            efms_df=pd.concat([efms_df,zero_efm_rows])

        #Reindex for consistency
        efms_df=efms_df.loc[original_rxn_ixs]

        #Filter        
        n_original=efms_df.shape[1]
        print(f'DONE... Computed {n_original} EFMs')
        for r in fluxes_to_enforce:
            efms_df=efms_df.loc[:,efms_df.loc[r]!=0.]
        n_final=efms_df.shape[1]

        #Transpose (It's easier to have differet EFMs being different rows, so that they can be better read in chunks)
        efms_df=efms_df.transpose()
        
        print(f'Removed {n_original-n_final} EFMs. {n_final} EFMs left after filtering.  Saving to file (this may take a while)')
        efms_df.to_csv(path_csv)
        print(f'Found {efms_df.shape[0]} EFMs for {carbon_source} under {condition} conditions')
        print('...........................................')
        print('\n\n\n')
        
        return {'S':S,'reversibilities':reversibilities,'reaction_ids':reactions_ids,'metabolite_ids':metabolite_ids},efms_df



# Enumerate EFMs (removing Oxygen-sensitive reactions from aerobic EFM enumeration)

In [6]:
carbon_sources=['glc__D','succ','fum','lac__D','ac','pyr' ,'glyc','rib__D','xyl__D','akg']
conditions=['aerobic','anaerobic']
carbon_source_specific_remove=None
condition_specific_remove={'aerobic':['DHORD5','PFL','FRD2']}

In [7]:
out={}
for substrate in tqdm.tqdm(carbon_sources):
    out[substrate]={}
    for condition in conditions:
        if condition=='aerobic':
            fluxes_to_enforce=['Biomass','EX_o2_e']
        elif condition=='anaerobic':
            fluxes_to_enforce=['Biomass']
        data,efms=\
        enumerate_efms(model,
                       substrate,
                       condition,
                       fluxes_to_enforce=fluxes_to_enforce,
                       carbon_source_specific_remove=carbon_source_specific_remove,
                       condition_specific_remove=condition_specific_remove,
                       )
        del efms

  0%|          | 0/10 [00:00<?, ?it/s]

Computing EFMs for glc__D aerobic....
Removed ['DHORD5', 'PFL', 'FRD2'] due to carbon source or condition specific parameter.
Starting EFMTools computation
2024-07-01  16:04:31.989  main                     INFO     | logger initialized
2024-07-01  16:04:31.990  main                     INFO     | =====================================================
2024-07-01  16:04:31.990  main                     INFO     | efmtool version 4.7.1, 2009-12-04 18:30:05
2024-07-01  16:04:31.990  main                     INFO     | Copyright (c) 2009, Marco Terzer, Zurich, Switzerland
2024-07-01  16:04:31.990  main                     INFO     | This is free software, !!! NO WARRANTY !!!
2024-07-01  16:04:31.990  main                     INFO     | See LICENCE.txt for redistribution conditions
2024-07-01  16:04:31.990  main                     INFO     | =====================================================
2024-07-01  16:04:32.121  main    efm.output.mat   INFO     | estimated efms-per-file: 810000
202

  0%|          | 0/10 [10:03<?, ?it/s]

Removed 58218 EFMs. 1035696 EFMs left after filtering.  Saving to file (this may take a while)


OSError: Cannot save file into a non-existent directory: 'efms'